# Conoscere l'Intelligenza Artificiale
***
## Lezione 2 - Complex Neural Networks and Overfitting


# Importiamo quello che ci serve

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Dropout
from keras.models import Sequential as Modello_Mele
from tensorflow.keras import layers
%pip install livelossplot
from livelossplot import PlotLossesKeras
TODO=0
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Carichiamo il Dataset

In [ ]:
dati_mela=pd.read_csv("https://raw.githubusercontent.com/alberto-rota/Conoscere-l-Intelligenza-Artificiale/main/mele.csv",index_col=0)

In [ ]:
dati_mela.head(TODO) # TODO

# Encoding

In [ ]:
dati_mela['Qualità']= dati_mela['Qualità'].map({"buona": 1,"non buona": 0}) if dati_mela['Qualità'].dtype == 'O' else dati_mela['Qualità']

In [ ]:
dati_mela.head(TODO) # TODO

# Visualizziamo i dati

In [ ]:
n_rows = 2
n_cols = int(np.ceil(len(dati_mela.columns) / n_rows))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))

for i, column in enumerate(dati_mela.columns):
    row = i // n_cols
    col = i % n_cols
    ax = axes[row, col]
    dati_mela[column].hist(ax=ax)
    ax.set_title(f'{column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Frequency')
plt.tight_layout()
plt.show()

# I dati sono OK?

# Ispezioniamo le relazioni tra i dati

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,15))
ax= plt.subplot(3,2,1)
dati_mela.plot(ax=ax, kind='scatter', x='Dimensione', y='Peso', s=32, alpha=0.5, c='Qualità',colormap='RdYlGn')
ax= plt.subplot(3,2,2)
dati_mela.plot(ax=ax, kind='scatter', x='Peso', y='Dolcezza', s=32, alpha=0.5, c='Qualità',colormap='RdYlGn')
ax= plt.subplot(3,2,3)
dati_mela.plot(ax=ax, kind='scatter', x='Dolcezza', y='Croccantezza', s=32, alpha=0.5, c='Qualità',colormap='RdYlGn')
ax= plt.subplot(3,2,4)
dati_mela.plot(ax=ax, kind='scatter', x='Croccantezza', y='Succosità', s=32, alpha=0.5, c='Qualità',colormap='RdYlGn')
ax= plt.subplot(3,2,5)
dati_mela.plot(ax=ax, kind='scatter', x='Succosità', y='Maturazione', s=32, alpha=0.5, c='Qualità',colormap='RdYlGn')
ax= plt.subplot(3,2,6)
dati_mela.plot(ax=ax, kind='scatter', x='Maturazione', y='Acidità', s=32, alpha=0.5, c='Qualità',colormap='RdYlGn')

# Controlliamo un altro scatterplot

In [ ]:
dati_mela.plot(kind='scatter', x='Dolcezza', y='Dimensione', s=32, alpha=0.5, c='Qualità',colormap='RdYlGn')
plt.show()

In [ ]:
target = dati_mela['Qualità']
unique, count = np.unique(target, return_counts=True)
print(f'Mele buone={count[0]}, Mele non buone={count[1]}')

# Normalizzazione

In [ ]:
dati_mela.describe()

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled=scaler.fit_transform(dati_mela)
dati_mela_minmax=pd.DataFrame(scaled)
X = dati_mela_minmax.iloc[:,:-1]
target = dati_mela_minmax.iloc[:,-1]
dati_mela_minmax.head(10)

In [ ]:
# Questa cella non fa niente ma DEVE essere eseguita
def draw_neural_net(model, left =.1, right=.9, bottom=.1, top=.9):
    '''
    Draw a neural network cartoon using matplotilb.

    :param ax: matplotlib.axes.Axes, the axes on which to plot the cartoon
    :param left: float, the center of the leftmost node(s) will be placed here
    :param right: float, the center of the rightmost node(s) will be placed here
    :param bottom: float, the center of the bottommost node(s) will be placed here
    :param top: float, the center of the topmost node(s) will be placed here
    :param layer_sizes: list of int, list containing the number of nodes in each layer
    '''
    layer_sizes = [model.input_shape[-1]]
    for layer in model.layers:
        # Check if layer is a Dense layer (ignoring input and output layers for simplicity)
        if isinstance(layer, tf.keras.layers.Dense):
            layer_sizes.append(layer.units)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    fig = plt.figure(figsize=(20,10))
    ax = fig.gca()
    ax.axis('off')
    ax.set_aspect('equal')
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color='w', ec='k', zorder=4)
            ax.add_artist(circle)
            # Annotation for the layer number
            if m == layer_size - 1:
                plt.text(n*h_spacing + left, layer_top - m*v_spacing - v_spacing*3, f'{layer_size} N',
                         ha='center', va='center')
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                  [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k',linewidth=0.25)
                ax.add_artist(line)
                plt.title("Rete Neurale")
    plt.show()

def reinitialize(model):
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))


In [ ]:
model = Modello_Mele()
LAYERS = [
    Input((7,)), # Input --> NON CAMBIARE!
    #
    Dense(TODO, activation='sigmoid'),
    Dense(TODO, activation='linear'),    # TODO: Cambiamo il numero di neuroni in ogni layer!
    Dense(TODO, activation='sigmoid'),
    #Dense(3, activation='linear'),
    #TODO: Aggiungiamo  layers come questi sopra. Quanti layer? Quanti neuroni? Quale activation function?
    #...
    Dense(1, activation='sigmoid')  # Output --> NON CAMBIARE!
]
for l in LAYERS:
    model.add(l)
draw_neural_net(model)

# Quanto complesso è il nostro modello?

In [ ]:
model.summary()

# Training!

In [ ]:
###################################################################
LEARNING_RATE = 1e-4
BATCH_SIZE    = 2          # TODO: Cambiare questi valori!
EPOCHE        = 200
###################################################################
reinitialize(model)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history=model.fit(
    X,target,
    batch_size=BATCH_SIZE,
    epochs=EPOCHE,
    callbacks=[PlotLossesKeras()],
    verbose=0
)

# Testiamo su dati nuovi

In [ ]:
dati_mela_test=pd.read_csv("https://raw.githubusercontent.com/alberto-rota/Conoscere-l-Intelligenza-Artificiale/main/mele_test.csv",index_col=0)
dati_mela_test.head(20)

# Normalizziamo

In [ ]:
scaled=scaler.fit_transform(dati_mela_test)
dati_mela_test_minmax=pd.DataFrame(scaled)
X_test = dati_mela_test_minmax.iloc[:,:-1]
target_test = dati_mela_test_minmax.iloc[:,-1]
dati_mela_test_minmax.head(10)


In [ ]:
predizioni = model.predict(X_test,verbose=False)
dati_mela_test["Predizione"] = predizioni
dati_mela_test.head(20)

# Classifichiamo
<img src=https://raw.githubusercontent.com/Codecademy/docs/main/media/sigmoid-function.png width="500">



In [ ]:
dati_mela_test["Predizione"] = predizioni > TODO  #TODO: Scegliamo una soglia!

dati_mela_test["Predizione"]=dati_mela_test["Predizione"].map({True:1,False:0})
dati_mela_test.head(20)

# Calcoliamo la nostra accuratezza

In [ ]:
test_accuracy = accuracy_score(dati_mela_test["Qualità"], dati_mela_test["Predizione"])*100
print(f'Accuratezza : {round(test_accuracy, 4)}%')

# Confusion Matrix

In [ ]:
confusionmatrix=confusion_matrix(dati_mela_test["Qualità"], dati_mela_test["Predizione"])
sns.heatmap(confusionmatrix,annot=True,fmt='g',xticklabels=['Predetta Buona ','Predetta Non buona'],yticklabels=['Buona ','Non Buona'])
plt.show()

In [ ]:
confusionmatrix=confusion_matrix(dati_mela_test["Qualità"], dati_mela_test["Qualità"])
sns.heatmap(confusionmatrix,annot=True,fmt='g',xticklabels=['Predetta Buona ','Predetta Non buona'],yticklabels=['Buona ','Non Buona'])
plt.show()

In [ ]:
newmodel = Modello_Mele()
LAYERS = [
    Input((7,)), # Input --> NON CAMBIARE!
    #
    #Dense(4, activation='sigmoid'),
    #Dense(3, activation='linear'),
    #TODO: Aggiungiamo  layers come questi sopra. Quanti layer? Quanti neuroni? Quale activation function?
    #...
    Dense(1, activation='sigmoid')  # Output --> NON CAMBIARE!
]
for l in LAYERS:
    newmodel.add(l)
draw_neural_net(newmodel)
# newmodel = model                      #TODO: Togliere asterisco se si vuole usare il modello nuovo

In [ ]:
###################################################################
LEARNING_RATE = 1e-3
BATCH_SIZE    = 16
EPOCHE        = 200
###################################################################
reinitialize(model)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics='accuracy'
)
history=model.fit(
    X,target,
    validation_data=(X_test,target_test),
    batch_size=BATCH_SIZE,
    epochs=EPOCHE,
    callbacks=[PlotLossesKeras()],
    verbose=0
)

In [ ]:
predizioni = model.predict(X_test,verbose=False)
dati_mela_test["Predizione"] = predizioni # TODO: Re-impostiamo la soglia
dati_mela_test["Predizione"]=dati_mela_test["Predizione"].map({True:1,False:0})
dati_mela_test.head(20)

In [ ]:
test_accuracy = accuracy_score(dati_mela_test["Qualità"], dati_mela_test["Predizione"])*100
print(f'Accuratezza : {round(test_accuracy, 4)}%')

In [ ]:
confusionmatrix=confusion_matrix(dati_mela_test["Qualità"], dati_mela_test["Predizione"])
sns.heatmap(confusionmatrix,annot=True,fmt='g',xticklabels=['Predetta Buona ','Predetta Non buona'],yticklabels=['Buona ','Non Buona'])
plt.show()

# Prediciamo Mele Nuove!

In [ ]:
dati_mela_nuovi=pd.read_csv("https://raw.githubusercontent.com/alberto-rota/Conoscere-l-Intelligenza-Artificiale/main/mele_nuove.csv",index_col=0)
dati_mela_nuovi.head(20)

In [ ]:
predizioni = model.predict(dati_mela_nuovi,verbose=False)
dati_mela_nuovi["Predizione"] = predizioni
dati_mela_nuovi.head(20)

In [ ]:
scaled=scaler.fit_transform(dati_mela_nuovi)
dati_mela_nuovi_minmax=pd.DataFrame(scaled)
X_nuovi = dati_mela_nuovi_minmax.iloc[:,:-1]
target_nuovi = dati_mela_nuovi_minmax.iloc[:,-1]
dati_mela_nuovi_minmax.head(10)

In [ ]:
predizioni = model.predict(X_nuovi,verbose=False)
dati_mela_nuovi["Predizione"] = predizioni
dati_mela_nuovi.head(20)

In [ ]:
dati_mela_nuovi["Predizione"] = predizioni > 0.5
dati_mela_nuovi["Predizione"]=dati_mela_nuovi["Predizione"].map({True:1,False:0})
dati_mela_nuovi.head(30)